In [315]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "dim_reduction"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    #print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.utils import io

with io.capture_output() as captured:
    print('no prints in here')

import time
import pandas as pd

In [316]:
Alone = 0
Spontan = 1
Sync = 2

First lets load the right hand solo,
and we would actually from the get go merge it to the solo hands

In [317]:
HandRight = pd.read_csv("Unity Data\HandRight.csv")
#HandRight.head(10)

In [318]:
drop_indecies = [1,2,3] # notice we are droping hands count because this is cheating!

In [319]:
HandRight = HandRight.drop(HandRight.columns[drop_indecies], axis=1)
HandRight = HandRight.iloc[500:4000]

In [320]:
HandRight.shape

#HandRight.info()

(3500, 19)

In [321]:
#HandRight.iloc[1::3,:]

In [322]:
def TransformData(df, type):
    df = df.drop(df.columns[drop_indecies], axis=1)
    if(type == Alone):
        df = df.iloc[500:4000]
        maxX = df.shape[0]
        combine = np.hstack([HandRight[:maxX].values, df.values]).reshape(-1, df.shape[1])
        df = pd.DataFrame(combine, columns=df.columns)
    else:
        df = df.iloc[1000:9000]
        maxX = int(df.shape[0] / 2) * 2
        df = df.iloc[:maxX]
    df["state"] = type
    return df

In [323]:
import glob

class DataLoader:
    def __init__(self, path):
        li = []
        all_files = glob.glob(path + "/*")
        v = 0
        for folder in all_files:
            print("\nloading in" ,folder, ':')
            files = glob.glob(folder + "/*.csv")
            for filename in files:
                df = pd.read_csv(filename, index_col=None, header=0)
                type = None
                if("Alone" in filename):
                    type = Alone
                elif("Sync" in filename):
                    type = Sync
                elif("Spontan" in filename):
                    type = Spontan
                df = TransformData(df, type)
                v += df.shape[0]
                li.append(df)
                print('loaded ', filename, type)
        self.dataRaw = li
        self.dataMerged = pd.concat(li, axis=0, sort=False)
        print(self.dataMerged.shape, v)

In [324]:
training = DataLoader('Unity Data\Training')


loading in Unity Data\Training\Evyatar Cohen :
loaded  Unity Data\Training\Evyatar Cohen\Evyatar636771052727603804Spontan.csv 1
loaded  Unity Data\Training\Evyatar Cohen\Evyatar636771053639929594Sync.csv 2
loaded  Unity Data\Training\Evyatar Cohen\Evyatar636771054555711409Alone.csv 0

loading in Unity Data\Training\Nofar Social_Nuero :
loaded  Unity Data\Training\Nofar Social_Nuero\Nofar636759795182793299Spontan.csv 1
loaded  Unity Data\Training\Nofar Social_Nuero\Nofar636759796290435160Alone.csv 0
loaded  Unity Data\Training\Nofar Social_Nuero\Nofar636759797397919664Sync.csv 2

loading in Unity Data\Training\Oriya Social_Nuero :
loaded  Unity Data\Training\Oriya Social_Nuero\Oriya636759804404113837Spontan.csv 1
loaded  Unity Data\Training\Oriya Social_Nuero\Oriya636759805268396661Alone.csv 0
loaded  Unity Data\Training\Oriya Social_Nuero\Oriya636759806131350399Sync.csv 2

loading in Unity Data\Training\Orya Kalmanovitz :
loaded  Unity Data\Training\Orya Kalmanovitz\OryaB6367710827366

In [325]:
data = pd.DataFrame
data = training.dataMerged

dataRaw is all the daraframes that we have already setup for us, now what we need is to generate from all of this,

our actuall training data

In [336]:
training.dataRaw[4].head(5)

,Time,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,state
0,136.6184,1.854658,163.3199,12.51509,67.33890,-6.28195,101.442200,-0.155174,0.078463,-0.764891,59.46534,173.0233,60.61472,254.6348,127.27250,240.62390,0.0,0.401652,0.0,0
1,504.3504,-20.808590,226.0860,17.08860,49.71883,-341.25160,-14.951780,1.199416,0.119953,1.519325,-92.19111,218.0329,12.57041,-309.3688,85.11283,-24.33749,0.0,0.603028,0.0,0
2,136.6354,2.990307,163.0106,14.22045,63.82683,-27.71448,91.912690,-0.166827,0.079468,-0.760001,60.39677,173.5084,62.45371,257.0284,134.33960,242.42510,0.0,0.405177,0.0,0
3,504.3669,-20.061710,220.5413,17.05443,36.02574,-309.41180,0.574942,1.089347,0.092826,1.515014,-91.69262,214.4963,13.19102,-314.4200,89.30956,-17.08960,0.0,0.313147,0.0,0
4,136.6524,4.197042,162.9949,15.69018,69.52037,12.69234,67.462470,-0.172697,0.079392,-0.758363,61.55299,174.0561,63.92721,259.6933,142.51660,243.74310,0.0,0.378955,0.0,0


Lets generate our np_array so first of all lets create a new class,

class would do the following it would get a settings : colloums it should drop, 

(jumps in time, and number of frames per row, andoverall jumps on the dataset),

In [327]:
class DataToNP:
    def __init__(self, drop_indecies, label_index, jumps):
        self.drop = drop_indecies
        self.label_index = label_index
        self.jumps = jumps
    def transform(self, df):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-327-34eacd1e77b6>, line 2)

In [328]:
data.shape
data.info()

(193128, 20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193128 entries, 1000 to 4499
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Time             193128 non-null  float64
 1    Position X      193128 non-null  float64
 2    Position Y      193128 non-null  float64
 3    Position Z      193128 non-null  float64
 4    Velocity X      193128 non-null  float64
 5    Velocity Y      193128 non-null  float64
 6    Velocity Z      193128 non-null  float64
 7    Pitch           193128 non-null  float64
 8    Roll            193128 non-null  float64
 9    Yaw             193128 non-null  float64
 10   Wrist Pos X     193128 non-null  float64
 11   Wrist Pos Y     193128 non-null  float64
 12   Wrist Pos Z     193128 non-null  float64
 13   Elbow pos X     193128 non-null  float64
 14   Elbow Pos Y     193128 non-null  float64
 15   Elbow Pos Z     193128 non-null  float64
 16   Grab Strenth    193128 non-null  flo

nice all the data, is complete with no nulls!

next step i wanna insert between

In [329]:
data.loc[data.state == Sync].values

array([[ 4.355143e+02, -3.878169e+01,  2.384981e+02, ...,  4.714104e-01,
         0.000000e+00,  2.000000e+00],
       [ 4.355143e+02, -1.471593e+02,  2.712473e+02, ...,  3.219693e-02,
         0.000000e+00,  2.000000e+00],
       [ 4.355313e+02, -3.310929e+01,  2.353336e+02, ...,  5.058866e-01,
         0.000000e+00,  2.000000e+00],
       ...,
       [ 8.562958e+02,  1.038475e+02,  2.147484e+02, ...,  0.000000e+00,
         0.000000e+00,  2.000000e+00],
       [ 8.563295e+02, -8.405124e+01,  1.878450e+02, ...,  0.000000e+00,
         0.000000e+00,  2.000000e+00],
       [ 8.563295e+02,  1.061188e+02,  2.180190e+02, ...,  0.000000e+00,
         0.000000e+00,  2.000000e+00]])

In [330]:
data

,Time,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,Roll,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,state
1000,344.2817,146.37590,177.4566,-34.298390,161.27960,-46.72000,28.471250,-0.907758,0.423413,-1.099889,202.0399,210.9716,-9.521991,359.1807,215.899800,187.13020,0.189790,1.473098,0.0,1
1001,344.2817,-91.63244,137.1377,-1.555785,125.18220,-147.96460,28.538970,0.609818,0.224510,1.254671,-159.3512,121.8911,11.383210,-364.5428,-20.581530,42.80880,0.000000,1.314333,0.0,1
1002,344.2983,147.60640,175.5879,-33.666630,48.28885,-111.12560,31.651890,-0.952355,0.478593,-1.083873,201.0957,212.4466,-8.900825,355.0660,228.117800,189.68710,0.250022,1.504737,0.0,1
1003,344.2983,-90.20468,135.4151,-1.015964,75.52746,-79.42178,32.457540,0.609856,0.212648,1.267614,-158.3547,121.4457,11.583490,-371.8665,-5.555285,52.48656,0.107532,1.470760,0.0,1
1004,344.3148,147.13200,172.9235,-33.415880,-50.32516,-153.80190,-7.015425,-0.987267,0.525830,-1.069649,198.5992,212.6112,-8.750634,348.8271,239.096300,191.54840,0.291985,1.536698,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,856.2631,101.91230,212.0264,42.535680,79.95810,113.41030,190.771400,1.065334,-1.225539,-0.574317,131.8590,149.6264,66.098530,283.6185,-58.034910,95.29868,0.000000,0.000000,0.0,2
4496,856.2958,-81.57359,183.5054,-7.185198,-63.24731,121.75270,62.959700,0.762549,1.824587,0.459446,-109.7250,130.7078,34.033600,-295.5957,-14.978170,140.02630,0.000000,0.000000,0.0,2
4497,856.2958,103.84750,214.7484,47.070820,61.11773,95.13531,155.745300,1.110287,-1.214715,-0.612753,134.0416,151.5924,68.176930,288.9688,-54.138200,94.23959,0.000000,0.000000,0.0,2
4498,856.3295,-84.05124,187.8450,-5.172269,-57.17249,109.20360,58.481460,0.824793,1.787295,0.434273,-109.5167,131.6489,33.329450,-282.9649,-31.354620,135.07610,0.000000,0.000000,0.0,2
